In [1]:
from qiskit import *
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import minimize
df = pd.read_csv (r'Data.csv')
shots = 100

In [2]:
Y = list(df['Y'])
Theta = list(df['Theta'])
State = list(df['State'])

In [15]:
Y_Train = Y[:2000]
T_Train = Theta[:2000]
S_Train = State[:2000]

Y_Test = Y[1000:]
T_Test = Theta[1000:]
S_Test = State[1000:]

In [17]:
df_1 = pd.DataFrame({'Y':Y_Train, 'Theta':T_Train,'State':S_Train,'z': [0]*len(Y_Train)})
fig = px.scatter_3d(df_1, x='Y', y='Theta',z='z', color='State')
fig.show()

# V1

In [138]:
def Circuit_v1(Params,V):
    qc = QuantumCircuit(2,2)
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    for i in range(int(len(Params)/3)):
        qc.ry(Params[0+3*i],0)
        qc.ry(Params[1+3*i],1)
        qc.cry(Params[2+3*i],0,1)
    qc.measure([0,1],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=500)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    Keymax = max(counts, key=counts.get) 
    return (int(Keymax,2))
    
        

In [139]:
Circuit_v1([0,0,0],[0,0])

1

In [134]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        if Circuit_v1(Params,[Y_Train[i],T_Train[i]]) != S_Train[i]:
            cost = cost+100
    return(cost)

In [143]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        if Circuit_v1(Params,[Y_Test[i],T_Test[i]])== S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [140]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(9,))

In [141]:
Cost(Params)

59000

In [142]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)

In [145]:
minimum.x

array([-2.37449503,  3.33540522,  0.40793613, -2.36240406, -1.10649642,
       -0.79583903, -0.66000836, -2.35014074, -1.42196828])

In [146]:
eff(minimum.x)

0.52375

In [147]:
eff(Params)

0.4045

# V4

In [225]:
def Circuit_v4(Params,V):
    qc = QuantumCircuit(2,2)
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    for i in range(int(len(Params)/3)):
        qc.ry(Params[0+3*i],0)
        qc.ry(Params[1+3*i],1)
        qc.cry(Params[2+3*i],0,1)
    qc.measure([0,1],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    output = [0]*4
    for i in counts.keys():
        output[int(i,2)] = counts[i]
    return (output)
    
        

In [226]:
Circuit_v4([0,0,0],[0,0])

[4, 47, 37, 12]

In [227]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        temp = [0]*4
        temp[S_Train[i]]= shots
        error = sum((np.array(Circuit_v4(Params,[Y_Train[i],T_Train[i]]))-np.array(temp))**2)
        cost = cost+error
    return(cost)

In [230]:
Params = [0]*30

In [231]:
Cost(Params)

6346628

In [232]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)

In [233]:
minimum.x

array([ 0.1170132 , -0.42036163,  0.55312232,  0.50268252,  0.09282682,
        1.84009056,  0.00678389,  1.05488069,  0.67253494,  0.68771706,
       -0.26923534,  1.07039248, -0.76318953, -0.28810543, -0.09133329,
        0.20235917,  0.23342814, -0.08794647, -0.14680746, -0.29787896,
       -0.00233571,  0.48069056, -0.31079169, -0.08701671, -0.15631617,
        0.21349515, -0.09512506, -0.50412323,  0.24445531, -0.10550188])

In [234]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v4(Params,[Y_Test[i],T_Test[i]])
        if output.index(max(output)) == S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [235]:
eff(minimum.x)

0.53375

In [236]:
eff(Params)

0.114

In [237]:
Params == minimum.x

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

# V2

In [59]:
def Circuit_v2(Params,V):
    qc =QuantumCircuit(8,2)
    #qc.h([0,1])
    #defining
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    #initializartion
    qc.ry(Params[0],2)
    qc.ry(Params[1],3)
    qc.ry(Params[2],4)
    qc.ry(Params[3],5)
    qc.ry(Params[4],6)
    qc.ry(Params[5],7)
    #network
    qc.cry(Params[6],0,2)
    qc.cry(Params[7],0,3)
    qc.cry(Params[8],0,4)
    qc.cry(Params[9],0,5)
    
    qc.cry(Params[10],1,2)
    qc.cry(Params[11],1,3)
    qc.cry(Params[12],1,4)
    qc.cry(Params[13],1,5)
    
    qc.cry(Params[14],2,6)
    qc.cry(Params[15],3,6)
    qc.cry(Params[16],4,6)
    qc.cry(Params[17],5,6)
    
    qc.cry(Params[18],2,7)
    qc.cry(Params[19],3,7)
    qc.cry(Params[20],4,7)
    qc.cry(Params[21],5,7)
    
    qc.measure([6,7],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=100)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    Keymax = max(counts, key=counts.get) 
    return (int(Keymax,2))
    #return (counts)
    
    
    
    
    
    

# V3

In [182]:
def Circuit_v3(Params,V):
    qc =QuantumCircuit(10,4)
    #qc.h(range(10))
    #defining
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    #initializartion
    qc.ry(Params[0],2)
    qc.ry(Params[1],3)
    qc.ry(Params[2],4)
    qc.ry(Params[3],5)
    qc.ry(Params[4],6)
    qc.ry(Params[5],7)
    qc.ry(Params[6],8)
    qc.ry(Params[7],9)
    #network
    # Layer1
    qc.cry(Params[8],0,2)
    qc.cry(Params[9],0,3)
    qc.cry(Params[10],0,4)
    qc.cry(Params[11],0,5)
    
    qc.cry(Params[12],1,2)
    qc.cry(Params[13],1,3)
    qc.cry(Params[14],1,4)
    qc.cry(Params[15],1,5)
    
    #Layer2
    
    qc.cry(Params[16],2,6)
    qc.cry(Params[17],3,6)
    qc.cry(Params[18],4,6)
    qc.cry(Params[19],5,6)
    
    qc.cry(Params[20],2,7)
    qc.cry(Params[21],3,7)
    qc.cry(Params[22],4,7)
    qc.cry(Params[23],5,7)
    
    qc.cry(Params[24],2,8)
    qc.cry(Params[25],3,8)
    qc.cry(Params[26],4,8)
    qc.cry(Params[27],5,8)
    
    qc.cry(Params[28],2,9)
    qc.cry(Params[29],3,9)
    qc.cry(Params[30],4,9)
    qc.cry(Params[31],5,9)
    
    qc.measure([6,7,8,9],[0,1,2,3])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    #counts = job_result.get_counts(qc)
    #Keymax = max(counts, key=counts.get) 
    
    
    A = list(job_result.get_counts(qc).keys())
    output = [0]*4
    for i in A:
        B = list(i)
        for j in range (4):
            if B[j] == '1':
                output[j]+= int(job_result.get_counts(qc).get(i)) 
                
    #result =output.index(max(output))
    #return (int(Keymax,2))
    #return (counts)
    #return(output)
    return(output)
    
    
    
    
    

In [183]:
Circuit_v3([0]*32,[0,0])

[0, 0, 0, 0]

In [185]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(32,))

In [186]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        temp = [0]*4
        #if Circuit_v3(Params,[Y_Train[i],T_Train[i]]) != S_Train[i]:
        temp[S_Train[i]] = shots
        error = sum((np.array(Circuit_v3(Params,[Y_Train[i],T_Train[i]]))-np.array(temp))**2)
        
        cost = cost+error
    return(cost)

In [187]:
Cost(Params)

339795289

In [189]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)

In [190]:
minimum

     fun: 176495220.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 327
  status: 1
 success: True
       x: array([ 2.36705472,  1.24926311,  4.59378538, -2.7593    , -1.00107435,
        1.52731879,  2.17640568, -0.27802212,  0.9989236 , -1.15852094,
        1.69318104, -3.20886846,  3.93442468,  0.17590778, -0.04670514,
        0.73294516, -0.33291347,  2.94175768,  0.88136078,  1.21260967,
       -1.44342593,  0.05330145, -3.06137338,  3.29094425,  3.8218712 ,
        2.59502231,  0.51668606,  1.14240523,  1.45290548,  2.19223313,
       -2.29590533, -0.66144903])

In [191]:
opt_params = minimum.x

In [192]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v3(Params,[Y_Test[i],T_Test[i]])
        if output.index(max(output)) == S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [193]:
eff(opt_params)

0.424

In [194]:
opt_params==Params

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

# V5

In [210]:
def Circuit_v5(Params,V):
    qc =QuantumCircuit(10,4)
    #qc.h(range(10))
    #defining
    qc.h(range(2))
    qc.x([6,7,8,9])
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    #initializartion
    qc.ry(Params[0],2)
    qc.ry(Params[1],3)
    qc.ry(Params[2],4)
    qc.ry(Params[3],5)
    qc.ry(Params[4],6)
    qc.ry(Params[5],7)
    qc.ry(Params[6],8)
    qc.ry(Params[7],9)
    #network
    # Layer1
    qc.cry(Params[8],0,2)
    qc.cry(Params[9],0,3)
    qc.cry(Params[10],0,4)
    qc.cry(Params[11],0,5)
    
    qc.cry(Params[12],1,2)
    qc.cry(Params[13],1,3)
    qc.cry(Params[14],1,4)
    qc.cry(Params[15],1,5)
    
    #Layer2
    
    qc.cry(Params[16],2,6)
    qc.cry(Params[17],3,6)
    qc.cry(Params[18],4,6)
    qc.cry(Params[19],5,6)
    
    qc.cry(Params[20],2,7)
    qc.cry(Params[21],3,7)
    qc.cry(Params[22],4,7)
    qc.cry(Params[23],5,7)
    
    qc.cry(Params[24],2,8)
    qc.cry(Params[25],3,8)
    qc.cry(Params[26],4,8)
    qc.cry(Params[27],5,8)
    
    qc.cry(Params[28],2,9)
    qc.cry(Params[29],3,9)
    qc.cry(Params[30],4,9)
    qc.cry(Params[31],5,9)
    
    qc.measure([6,7,8,9],[0,1,2,3])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    #counts = job_result.get_counts(qc)
    #Keymax = max(counts, key=counts.get) 
    
    
    A = list(job_result.get_counts(qc).keys())
    output = [0]*4
    for i in A:
        B = list(i)
        for j in range (4):
            if B[j] == '1':
                output[j]+= int(job_result.get_counts(qc).get(i)) 
                
    #result =output.index(max(output))
    #return (int(Keymax,2))
    #return (counts)
    #return(output)
    return(output)
    
    
    
    
    

In [204]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(32,))

In [211]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        temp = [0]*4
        #if Circuit_v3(Params,[Y_Train[i],T_Train[i]]) != S_Train[i]:
        temp[S_Train[i]] = shots
        error = sum((np.array(Circuit_v5(Params,[Y_Train[i],T_Train[i]]))-np.array(temp))**2)
        
        cost = cost+error
    return(cost)

In [206]:
Cost(Params)

124161898

In [212]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v5(Params,[Y_Test[i],T_Test[i]])
        if output.index(max(output)) == S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [208]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)
minimum

     fun: 83187947.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 297
  status: 1
 success: True
       x: array([-1.17940336, -0.99625212,  2.49206744, -1.60345853,  2.17923355,
       -0.56045746,  0.19277923,  0.18152768, -2.0938287 , -1.44598133,
       -0.54407588, -1.38023601,  3.44301152, -1.0200123 ,  1.01193131,
       -0.52851076, -2.19379001, -1.61131395, -2.54702458, -1.34440165,
        0.92128988,  2.03891945, -1.29425808,  1.61348226, -0.60576502,
        1.73073453,  0.37059438,  2.22592784, -0.41506344,  1.98927409,
        0.41599016,  0.30098711])

In [209]:
eff(minimum.x)

0.37825

# V6

In [219]:
def Circuit_v6(Params,V):
    qc =QuantumCircuit(10,4)
    #qc.h(range(10))
    #defining
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    #initializartion
    qc.ry(Params[0],2)
    qc.ry(Params[1],3)
    qc.ry(Params[2],4)
    qc.ry(Params[3],5)
    qc.ry(Params[4],6)
    qc.ry(Params[5],7)
    qc.ry(Params[6],8)
    qc.ry(Params[7],9)
    #network
    # Layer1
    qc.cry(Params[8],0,2)
    qc.cry(Params[9],0,3)
    qc.cry(Params[10],0,4)
    qc.cry(Params[11],0,5)
    
    qc.cry(Params[12],1,2)
    qc.cry(Params[13],1,3)
    qc.cry(Params[14],1,4)
    qc.cry(Params[15],1,5)
    
    qc.x([0,1])
    
    qc.cry(Params[16],0,2)
    qc.cry(Params[17],0,3)
    qc.cry(Params[18],0,4)
    qc.cry(Params[19],0,5)
    
    qc.cry(Params[20],1,2)
    qc.cry(Params[21],1,3)
    qc.cry(Params[22],1,4)
    qc.cry(Params[23],1,5)
    
    qc.x([0,1])
    
    
    
    
    
    #Layer2
    
    
    qc.cry(Params[24],2,6)
    qc.cry(Params[25],3,6)
    qc.cry(Params[26],4,6)
    qc.cry(Params[27],5,6)
    
    qc.cry(Params[28],2,7)
    qc.cry(Params[29],3,7)
    qc.cry(Params[30],4,7)
    qc.cry(Params[31],5,7)
    
    qc.cry(Params[32],2,8)
    qc.cry(Params[33],3,8)
    qc.cry(Params[34],4,8)
    qc.cry(Params[35],5,8)
    
    qc.cry(Params[36],2,9)
    qc.cry(Params[37],3,9)
    qc.cry(Params[38],4,9)
    qc.cry(Params[39],5,9)
    
    
    qc.x([2,3,4,5])
    
    qc.cry(Params[40],2,6)
    qc.cry(Params[41],3,6)
    qc.cry(Params[42],4,6)
    qc.cry(Params[43],5,6)
    
    qc.cry(Params[44],2,7)
    qc.cry(Params[45],3,7)
    qc.cry(Params[46],4,7)
    qc.cry(Params[47],5,7)
    
    qc.cry(Params[48],2,8)
    qc.cry(Params[49],3,8)
    qc.cry(Params[50],4,8)
    qc.cry(Params[51],5,8)
    
    qc.cry(Params[52],2,9)
    qc.cry(Params[53],3,9)
    qc.cry(Params[54],4,9)
    qc.cry(Params[55],5,9)
        
    qc.x([2,3,4,5])
    
    qc.measure([6,7,8,9],[0,1,2,3])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    #counts = job_result.get_counts(qc)
    #Keymax = max(counts, key=counts.get) 
    
    
    A = list(job_result.get_counts(qc).keys())
    output = [0]*4
    for i in A:
        B = list(i)
        for j in range (4):
            if B[j] == '1':
                output[j]+= int(job_result.get_counts(qc).get(i)) 
                
    #result =output.index(max(output))
    #return (int(Keymax,2))
    #return (counts)
    #return(output)
    return(output)
    
    
    
    
    

In [218]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(56,))

In [222]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        temp = [0]*4
        #if Circuit_v3(Params,[Y_Train[i],T_Train[i]]) != S_Train[i]:
        temp[S_Train[i]] = shots
        error = sum((np.array(Circuit_v6(Params,[Y_Train[i],T_Train[i]]))-np.array(temp))**2)
        
        cost = cost+error
    return(cost)
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v6(Params,[Y_Test[i],T_Test[i]])
        if output.index(max(output)) == S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [221]:
Cost(Params)

6016746

In [223]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)
minimum

     fun: 3390524.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 431
  status: 1
 success: True
       x: array([ 1.52186625e+00,  1.07009124e+00, -1.18522405e+00, -1.82549820e+00,
        1.47819887e+00,  1.48425581e+00,  1.80006799e+00, -1.03625126e+00,
        5.52516537e-01,  3.38731598e+00, -1.03521920e+00,  2.72935509e+00,
        1.14658289e+00,  1.78270886e+00, -2.31471839e-04, -1.08151529e+00,
       -2.38504460e+00, -3.08023619e-01,  3.76418940e-01,  2.10720821e+00,
       -1.84174269e+00,  2.12748202e+00, -5.88143984e-01, -6.13082114e-01,
       -9.91212267e-01, -3.06381882e-01,  1.64293958e+00, -1.81689109e+00,
       -9.95519895e-01, -8.12886649e-01,  1.01429443e-01,  2.15002528e+00,
        9.73858598e-01,  1.88687030e+00, -5.81340864e-01,  1.21293741e+00,
        1.20329736e+00,  2.14975320e+00,  3.69669005e-01,  2.42126891e+00,
        4.21011318e-02, -1.11205298e+00,  6.82675069e-01, -3.84375733e-01,
        3.04628218e-01, -1.97624364e+00,

In [224]:
eff(minimum.x)

0.3845

# V7

In [18]:
def Circuit_v7(Params,V):
    qc = QuantumCircuit(5,2)
    qc.h(range(3))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.ccx(0,1,2)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),2)
    qc.ccx(0,1,2)
    for i in range(int(len(Params)/10)):
        qc.ry(Params[0+10*i],0)
        qc.ry(Params[1+10*i],1)
        qc.ry(Params[2+10*i],2)
        qc.ry(Params[3+10*i],3)
        qc.ry(Params[4+10*i],4)
        
        
        
        qc.cry(Params[5+10*i],0,1)
        qc.cry(Params[6+10*i],1,2)
        qc.cry(Params[7+10*i],2,3)
        qc.cry(Params[8+10*i],3,4)
        qc.cry(Params[9+10*i],4,0)
        
        
        
    qc.measure([3,4],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    output = [0]*4
    for i in counts.keys():
        output[int(i,2)] = counts[i]
    return (output)
    
        

In [19]:
Circuit_v7([0]*10,[0,0])

[100, 0, 0, 0]

In [20]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        temp = [0]*4
        temp[S_Train[i]]= shots
        error = sum((np.array(Circuit_v7(Params,[Y_Train[i],T_Train[i]]))-np.array(temp))**2)
        cost = cost+error
    return(cost)
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        output = Circuit_v7(Params,[Y_Test[i],T_Test[i]])
        if output.index(max(output)) == S_Test[i]:
            cost = cost+1
    return(cost/len(Y_Test))

In [21]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(50,))

In [22]:
Cost(Params)

16342158

In [ ]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)
minimum

In [ ]:
eff(minimum.x)

In [5]:
params = [ 1.63341838, -0.90958667, -1.72936098,  1.30422852, -1.12632053,
        2.67215422,  0.59444125, -0.17382798, -1.17423312, -2.29277681,
       -1.44694606, -0.19132074, -2.6420889 , -1.62276693,  1.70550897,
       -0.65490977, -1.80780424, -0.61716655,  1.13028498, -2.5569678 ,
        1.21482219,  2.26495486, -2.31847892, -2.04575853, -2.53881896,
       -1.13470588,  0.27146519,  1.74428615,  2.18065488, -1.57295776,
        1.79040654,  1.11755875, -1.25724715, -1.01628673, -1.99751049,
        0.09622231, -2.39710637, -0.60231718, -0.83326536, -1.93454945,
       -1.13330953, -2.21606853, -0.18823943, -1.99578546, -2.13218772,
        0.36272045,  0.11469117, -2.80050014, -2.62200597,  1.7491935 ]

In [10]:
x_1=[]
y_1 =[]
z_1 =[]
x_2=[]
y_2=[]
z_2=[]


for i in range (len(Y_Test)):
    output = Circuit_v7(params,[Y_Test[i],T_Test[i]])
    if output.index(max(output)) == S_Test[i]:
        x_1.append(Y_Test[i])
        y_1.append(T_Test[i])
        z_1.append(S_Test[i])
    else:
        x_2.append(Y_Test[i])
        y_2.append(T_Test[i])
        z_2.append(S_Test[i])

In [11]:
df_2 = pd.DataFrame({'Y':x_1, 'Theta':y_1,'State':z_1,'z': [0]*len(x_1)})
fig = px.scatter_3d(df_2, x='Y', y='Theta',z='z', color='State')
fig.show()

In [12]:
df_3= pd.DataFrame({'Y':x_2, 'Theta':y_2,'State':z_2,'z': [0]*len(x_2)})
fig = px.scatter_3d(df_3, x='Y', y='Theta',z='z', color='State')
fig.show()

In [13]:
len(x_1)

1954

In [14]:
len(x_2)

2046

In [11]:
#! /usr/bin/python3

import sys
import pennylane as qml
import numpy as np
from scipy.optimize import minimize
import math

def classify_data(X_train, Y_train, X_test):
    #print("sdfsdf")
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        X_train (np.ndarray): An array of floats of size (250, 3) to be used as training data.
        Y_train (np.ndarray): An array of size (250,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (50, 3) to serve as testing data.

    Returns:
        str: The predicted categories of X_test, converted from a list of ints to a
            comma-separated string.
    """

    # Use this array to make a prediction for the labels of the data in X_test
    predictions = []
    # QHACK #
    dev = qml.device("default.qubit", wires=3, shots=1)
    @qml.qnode(dev)
    def circuit1(data,params):
        for i in range(3):
            qml.Hadamard(wires = i)
            qml.RY(data[i],wires = i)



        def entangling_layer(nqubits):

            for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
                qml.CNOT(wires=[i, i + 1])
            for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
                qml.CNOT(wires=[i, i + 1])
            '''
        qml.CZ(wires=[0,1])
        qml.RY((data[0]+data[1])/2,wires = 1)
        qml.CZ(wires=[0,1])
        qml.CZ(wires=[0,2])
        qml.RY((data[0]+data[2])/2,wires = 2)
        qml.CZ(wires=[0,2])
        qml.CZ(wires=[2,1])
        qml.RY((data[2]+data[1])/2,wires = 1)
        qml.CZ(wires=[2,1])
        '''
        #parameters = params.reshape((2, 3, 3))
        #qml.templates.StronglyEntanglingLayers(parameters, wires=range(3))
        #entangling_layer(3)
        '''
        for j in range(3):
            qml.RY(params[j],wires = j)
        '''
        for i in range(1):
            for j in range(3):
                qml.RY(params[i*3+j],wires = j)
                
            qml.CNOT(wires = [0,1])
            qml.CNOT(wires = [0,2])
            qml.CNOT(wires = [1,2])
            qml.CNOT(wires = [2,0])
            for j in range(3):
                qml.RY(params[i*3+3+j],wires = j)
            qml.CNOT(wires = [0,1])
            qml.CNOT(wires = [0,2])
            qml.CNOT(wires = [1,2])
            qml.CNOT(wires = [2,0])

        '''
        qml.CRY(params[0],wires = [0,1])
        qml.CRY(params[1],wires = [0,2])
        qml.CRY(params[2],wires = [1,2])
        qml.CRY(params[3],wires = [2,0])
        qml.CRY(params[4],wires = [0,1])
        qml.CRY(params[5],wires = [1,0])
        qml.CRY(params[6],wires = [1,2])
        qml.CRY(params[7],wires = [2,0])
        '''
        return qml.expval(qml.PauliZ(0) )#@ qml.PauliZ(1))
        #return qml.probs(wires=(range(3)))
    params = [0,0,0,0,0,0]
    # QHACK #
    #print( Y_train)
    def pred(data,params):
        z = circuit1(data,params)
        '''
        #print(z)
        x = 0
        y= 0
        
        
        for i in range (2**3):
            if z[i]>y:
                y = z[i]
                x = i
        #print(x)
        
        if x<3:
            x = -1
        elif 2<x<6:
            x = 0
        else:
            x = 1
        
        return x
        
        '''
        z = math.floor(z*3-1)
        return z
    
    def cost(params):
        cost = 0
        for i in range (250):
            z = math.floor(circuit1(X_train[i],params)*3-1)
            #o = pred(X_train[i],params)
            #print(pred(X_train[i],params))
            if z != Y_train[i]:
                cost +=100
        return cost
    
    #print(predictions)
    #print(cost(params))
    '''
    opt = qml.AdagradOptimizer(stepsize=0.5)
    steps = 30
    for i in range(steps):
        params = opt.step(cost, params)
    '''
    minimum = minimize(cost, params, method="COBYLA", tol=0.01)
    #print(minimum.x)
    opt_params = minimum.x
    for i in range (50):
        predictions.append(pred(X_test[i],opt_params))
    
    

    #print(cost(opt_params))
    #print(opt_params)
    Try = [1,0,-1,0,-1,1,-1,-1,0,-1,1,-1,0,1,0,-1,-1,0,0,1,1,0,-1,0,0,-1,0,-1,0,0,1,1,-1,-1,-1,0,-1,0,1,0,-1,1,1,0,-1,-1,-1,-1,0,0]
    a =0
    
    for i in range(50):
        if Try[i]==predictions[i]:
            a = a+1
    
    #print(a)
    #print(predictions==Try)
    #print(cost(opt_params))
    return array_to_concatenated_string(predictions)


def array_to_concatenated_string(array):
    """DO NOT MODIFY THIS FUNCTION.

    Turns an array of integers into a concatenated string of integers
    separated by commas. (Inverse of concatenated_string_to_array).
    """
    return ",".join(str(x) for x in array)


def concatenated_string_to_array(string):
    """DO NOT MODIFY THIS FUNCTION.

    Turns a concatenated string of integers separated by commas into
    an array of integers. (Inverse of array_to_concatenated_string).
    """
    return np.array([int(x) for x in string.split(",")])


def parse_input(giant_string):
    """DO NOT MODIFY THIS FUNCTION.

    Parse the input data into 3 arrays: the training data, training labels,
    and testing data.

    Dimensions of the input data are:
      - X_train: (250, 3)
      - Y_train: (250,)
      - X_test:  (50, 3)
    """
    X_train_part, Y_train_part, X_test_part = giant_string.split("XXX")

    X_train_row_strings = X_train_part.split("S")
    X_train_rows = [[float(x) for x in row.split(",")] for row in X_train_row_strings]
    X_train = np.array(X_train_rows)

    Y_train = concatenated_string_to_array(Y_train_part)

    X_test_row_strings = X_test_part.split("S")
    X_test_rows = [[float(x) for x in row.split(",")] for row in X_test_row_strings]
    X_test = np.array(X_test_rows)

    return X_train, Y_train, X_test


if __name__ == "__main__":
    # DO NOT MODIFY anything in this code block

    X_train, Y_train, X_test = parse_input(sys.stdin.read())
    output_string = classify_data(X_train, Y_train, X_test)
    print(f"{output_string}")


ValueError: not enough values to unpack (expected 3, got 1)